In [16]:
# 📊 FC25 Sentiment Predictor Notebook

# ✅ STEP 1: Upload Required Files
from google.colab import files
uploaded = files.upload()

# Upload:
# - model.pkl
# - vectorizer.pkl
# - raw_comments.csv

# ✅ STEP 2: Load Model and Vectorizer
import pickle
import pandas as pd

model_file = [f for f in uploaded if f.startswith("model") and f.endswith(".pkl")][0]
vectorizer_file = [f for f in uploaded if f.startswith("vectorizer") and f.endswith(".pkl")][0]
comment_file = [f for f in uploaded if f.endswith(".csv")][0]

with open(model_file, "rb") as f:
    model = pickle.load(f)

with open(vectorizer_file, "rb") as f:
    vectorizer = pickle.load(f)

# ✅ STEP 3: Load and Clean Raw Comments
df = pd.read_csv(comment_file)
if df.columns[0] != "Comment":
    df.columns = ["Comment"]

df = df.drop_duplicates()
df = df[df["Comment"].str.strip() != ""]

# ✅ STEP 4: Predict Sentiment with Confidence Threshold
X_new = vectorizer.transform(df["Comment"])
probs = model.predict_proba(X_new)
preds = model.predict(X_new)
max_probs = probs.max(axis=1)

confidence_threshold = 0.60
final_labels = [label if prob >= confidence_threshold else "Uncertain"
                for label, prob in zip(preds, max_probs)]

df["Sentiment"] = final_labels
df = df[df["Sentiment"] != "Uncertain"]  # Discard uncertain predictions

# ✅ STEP 5: Export Labeled Comments
output_path = "labeled_comments.csv"
df.to_csv(output_path, index=False)
files.download(output_path)


('FC25_Trainer_GDrive.ipynb', 'FC25_Predictor_GDrive.ipynb')